In [21]:
import pandas as pd

games_info = pd.read_csv("C:\\Users\\dijan\\Documents\\GitHub\\Reccomender-system-for-board-games\\data\\raw\\games_detailed_info2025.csv")
games_info.rename(columns={'id': 'ID'}, inplace=True)

In [22]:
bgg_reviews = pd.read_csv("C:\\Users\\dijan\\Documents\\GitHub\\Reccomender-system-for-board-games\\data\\raw\\bgg-26m-reviews.csv")

In [23]:
import data_prep
game_features = [ 'yearpublished']
data_preprocessor = data_prep.DataPreprocessor(games_info, bgg_reviews)
game_info_cut = data_preprocessor.discretize_column(games_info, 'usersrated', n_bins=10, percent=0.25, drop_original=True )
game_info_final = data_preprocessor.discretize_column(game_info_cut, 'yearpublished', n_bins=10, drop_original=True)

game_merged = data_preprocessor.merge_datasets(featires_to_add=game_features)

game_info_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6945 entries, 0 to 6944
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     6945 non-null   int64  
 1   type                           6945 non-null   object 
 2   ID                             6945 non-null   int64  
 3   thumbnail                      6945 non-null   object 
 4   image                          6945 non-null   object 
 5   alternate                      4732 non-null   object 
 6   description                    6945 non-null   object 
 7   minplayers                     6945 non-null   int64  
 8   maxplayers                     6945 non-null   int64  
 9   suggested_num_players          6945 non-null   object 
 10  suggested_playerage            6945 non-null   object 
 11  suggested_language_dependence  6945 non-null   object 
 12  playingtime                    6945 non-null   int64 

In [24]:

game_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26195778 entries, 0 to 26195777
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Unnamed: 0     int64  
 1   user           object 
 2   rating         float64
 3   comment        object 
 4   ID             int64  
 5   name           object 
 6   yearpublished  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 1.4+ GB


In [9]:
game_info_cut.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6945 entries, 0 to 6944
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     6945 non-null   int64  
 1   type                           6945 non-null   object 
 2   ID                             6945 non-null   int64  
 3   thumbnail                      6945 non-null   object 
 4   image                          6945 non-null   object 
 5   alternate                      4732 non-null   object 
 6   description                    6945 non-null   object 
 7   yearpublished                  6945 non-null   int64  
 8   minplayers                     6945 non-null   int64  
 9   maxplayers                     6945 non-null   int64  
 10  suggested_num_players          6945 non-null   object 
 11  suggested_playerage            6945 non-null   object 
 12  suggested_language_dependence  6945 non-null   object